In [1]:
from __future__ import absolute_import, division, print_function
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import os
import re
import scipy.io

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

In [2]:
def blockshaped(arr, size):
    """
    Breaks an array into smaller pieces.  Returns an array of shape (n, nrows, ncols) where
    n * size * size = arr.size

    If arr is a 2D array, the returned array should look like n subblocks with
    each subblock preserving the "physical" layout of arr.
    
    Inputs
    arr: 2D Array to be split
    size: Size that split 2D arrays should be
    """
    h, w = arr.shape
    return (arr.reshape(h//size, size, -1, size)
               .swapaxes(1,2)
               .reshape(-1, size, size))

def crop_center(img,crop):
    """
    Crops an image into a square from the center outwards.
    
    Inputs
    img: 2D Array to be cropped
    crop: size to crop image to
    """
    # Obtain the shape of the image
    x, y, c = img.shape
    
    # The starting coordinates of the new image
    startx = x//2 - crop//2
    starty = y//2 - crop//2    
    return img[starty : starty + crop, startx : startx + crop, :]

In [3]:
# Optimized Data Extractor
crop_size = 1000
split_size = 100

# Define directory with matlab files
direc = 'D:/MLdata/18-01-09d-Exp/Extract Data/'

# Extract file names
files = [filename for filename in os.listdir(direc)][:10]

# Intialize Lists
split_images = []
Fn_label = []
Fs_label = []
T_label = []
log_T_label = []
block_label = []
exp_label = []

for file in files:
    print(file, end="\r")
    # Import Matlab data
    data = scipy.io.loadmat(direc + file)

    # Get data values from Matlab file
    Fn = data['Fn'].flatten()
    Fs = data['Fs'].flatten()
    T = data['T'].flatten()
    images = data['images'].astype(np.float)

    # Pull block and experiment values from filename
    block = np.repeat(int(re.search('block(.*)_', file).group(1)), len(T))
    exp = np.repeat(int(re.search('Exp(.*).mat', file).group(1)), len(T))
    
    # Crop images to appropriate size
    cropped_images = crop_center(images, crop_size)

    for i in range(cropped_images.shape[2]):
        #Break images into smaller pieces
        #Divide by 255 to normalize data
        temp = blockshaped(cropped_images[:,:,i], split_size)/255.0
        split_images.extend(temp)
    
    # Append data to lists
    # Adjust labels to account for the splitting of images
    Fn_label.extend(np.repeat(Fn, (crop_size/split_size)**2))
    Fs_label.extend(np.repeat(Fs, (crop_size/split_size)**2))
    T_label.extend(np.repeat(T, (crop_size/split_size)**2))
    log_T_label.extend(np.repeat(np.log(T), (crop_size/split_size)**2))
    block_label.extend(np.repeat(block, (crop_size/split_size)**2)) 
    exp_label.extend(np.repeat(exp, (crop_size/split_size)**2))
    
# Convert final results into arrays
split_images = np.array(split_images)
Fn_label = np.array(Fn_label)
Fs_label = np.array(Fs_label)
T_label = np.array(T_label)
log_T_label = np.array(log_T_label)
block_label = np.array(block_label) 
exp_label = np.array(exp_label)

In [4]:
# Assemble targets, shuffle data, and assign training and testing sets
targets = np.vstack((log_T_label,Fn_label,Fs_label)).T

# Obtain indicies for the items
ind = np.arange(split_images.shape[0])
# Shuffel the indices
np.random.shuffle(ind)

# Re-arrange the data using the shuffled indices
data = split_images[ind,:,:]
targets = targets[ind,:]

# Split train and test
# Set to use 80% of data for training
train_size = round(0.8*data.shape[0])

# Seperate Data
train_data = data[:train_size,:,:]
test_data = data[train_size:,:,:]

#Seperate Labels
train_labels = targets[:train_size,:]
test_labels = targets[train_size:,:]

In [5]:
def build_cnn():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(16, kernel_size=(8, 8), strides=(1, 1),
                     activation='tanh',
                     input_shape=(100,100,1)))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(keras.layers.Conv2D(32, (8, 8), activation='tanh'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(3))
    
    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
    
    return (model)

In [6]:
model = build_cnn()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [7]:
train_data = train_data.reshape(train_data.shape[0], train_data.shape[1], train_data.shape[2], 1)
test_data = test_data.reshape(test_data.shape[0], test_data.shape[1], test_data.shape[2], 1)

model.fit(train_data, train_labels, epochs=5)
predictions=model.predict(test_data)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
11392/27120 [===========>..................] - ETA: 4:11 - loss: 109.8102 - mean_absolute_error: 6.6694 - mean_squared_error: 109.8102

KeyboardInterrupt: 